# Imports
Import python libraries

In [19]:
import os

# Set dataset parameters
Choose the attributes to load the dataset.

In [20]:


detectors = ['mtcnn_serfiq']  # ,'dlib', 'opencv', 'mediapipe','mtcnn','mtcnn_serfiq'
embedding_models = ['QMagFace_SR','QMagFace']  # 'QMagFace', 'QMagFace_SR',"ArcFace","Dlib", 'VGG-Face','FaceVACs', 'Facenet512','ArcFace_normalized'
quality_models = ['ser_fiq'] #  'ser_fiq','tface'


#embedding_models = ['Facenet512','QMagFace','ArcFace'] 
#quality_models = ['ser_fiq','tface']

# Set LR parameters
Calibration, image filters, face filters, quality filters...

## Calibration

In [21]:
calibrators = ['logit']  # 'KDE', , 'isotonic','logit'

#calibration_db0 = ['XQLFW','SCFace'] # "SCFace",'ForenFace','ENFSI','ForenFace','LFW','SCFace','UTKface','XQLFW',
#image_filters0 = ['yaw', 'gender', 'race','emotion']   # 'yaw','pitch','roll', E.g. 'pose','gender','emotion','race'

calibration_db = ['ForenFace'] # "SCFace",'ForenFace','ENFSI','ForenFace','LFW','SCFace','UTKface','XQLFW','UTKface','ENFSI',
#'XQLFW','UTKface',"SCFace",'ForenFace',

image_filters = ['gender','yaw', 'pitch', 'race']   # 'yaw','pitch','roll', E.g. 'pose','gender','emotion','race'
# gender, yaw, pitch, race

face_image_filters = []  # 'confusion_score'
quality_filters = [] # 'quality_group'

#all the filters in one dictionary
filters = {'image': image_filters, 'face_image': face_image_filters,'quality_image': quality_filters}

n_calibration_pairs = 10000



## Validation

In [22]:
enfsi_years = [2015]

# Video parameters

In [23]:
import numpy as np
#quality_dropout = [0.01, 0.02, 0.05, 0.10, 0.20, 0.5, 1]
quality_dropout = list(np.linspace(0.1,1.0,10))

## Other parameters
Rest of the configuration

In [24]:
metrics = 'euclidean_l2'
embedding_model_as_scorer = True

# Scenarios
Different filters to do computations

In [25]:
scenario0 = []
scenario1 = ['gender', 'age']
scenario2 = ['yaw', 'pitch']
scenario3 = ['glasses', 'beard']
scenario4 = ['low_quality']
scenario5 = ['headgear']
scenario6 = ['terhorst_quality_rounded']

# Input and output directories
Where the sql database is and which folder the results are saved.

In [26]:
home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database')
db_name = 'colab_main_dataset_21b'

# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'SuperResolution')

# Preprocess data
Connect to sql database.

In [27]:
from sql_face.alchemy import get_session
from sql_face.tables import *

In [28]:
session = get_session(input_dir, db_name)

# Experiments

All parameters are specified in ExperimentalSetup, that creates a different experiment with all the posible combinations of detectors, embedding_models, calibrators, etc.

In [29]:
from lr_video_face.experiments import ExperimentalSetup

In [30]:
experimental_setup = ExperimentalSetup(detectors, 
                                        embedding_models, 
                                        quality_models,
                                        calibrators, 
                                        calibration_db, 
                                        enfsi_years,
                                        filters,
                                        metrics, 
                                        n_calibration_pairs,
                                        embedding_model_as_scorer,
                                        output_dir,
                                        session,
                                        quality_dropout)

# Results

In [31]:
'''for exp in experimental_setup:
    exp.perform()
    break '''

'for exp in experimental_setup:\n    exp.perform()\n    break '

In [32]:
from lr_video_face.evaluators import GlobalEvaluator  

In [33]:
global_results = GlobalEvaluator(experimental_setup)    

  0%|          | 0/2 [00:00<?, ?it/s]

## Individual plots

In [ ]:
global_results.make_experiment_plots()

## General plot

In [ ]:
# import pickle
# with open('data2.db','wb') as outp:
#     pickle.dump(global_results,outp,-1)

In [ ]:
global_results.make_global_plot()